# Demo: Chirp mass distribution of binary black holes (BBHs)

This notebook reproduces Figure X of the COMPAS Paper (_arXiv_: ; Mandel et al., in prep.). Before running the notebook, you need to download the COMPAS data (COMPAS_CHE_30xZ_3000000.h5) that was used to produce the Figure from the link https://drive.google.com/file/d/1gqEpV8B_JSQ30K1_aNb4B23csdgQEpfc/view?usp=sharing (MIKE:put on Zenodo?). Place the file in `$COMPAS_ROOT_DIR/docs/demo`.

This file contains the output of 3,000,000 binaries evolved using COMPAS v02.11.01a (v02.11.01 plus fix for issue #260) with the Fidicuial Model (Section X, Mandel et al., in prep.) over 30 different metallicities, uniform in $\log(Z) \in [10^{-4},10^{-1.825}]$. More details can be found in `Run_Details.txt`.

We include some comments to clarify different parts of the code below. But for a full exposition to COMPAS's post-processing tools, please work through the Jupyter Notebook tutorials in the `postProcessing` directory (MIKE: include link).

## Paths and libraries
You may have to make several changes here for the code to run:
* `pathData` needs to be set to the directory containing the COMPAS_CHE_30xZ_3000000.h5 file you have downloaded. Currently, we set this path to be the demo directory.
* If you would like to use this script for your own COMPAS Output, you will need to change `fileName` as well.

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

COMPASdir = os.environ['COMPAS_ROOT_DIR']
pathScripts = COMPASdir + "/postProcessing/Folders/CosmicIntegration/PythonScripts/"
pathData = COMPASdir + "/docs/demo/"
fileName = "COMPAS_CHE_30xZ_3000000.h5"

sys.path.append(pathScripts)
import ClassCosmicIntegrator

# Importing ClassCosmicIntegrator imports other classes, including ClassCOMPAS,
# ClassMSSFR, selection_effects, totalMassEvolvedPerZ. 

$HOME/git/COMPAS//postProcessing/Folders/CosmicIntegration/PythonScripts/


ModuleNotFoundError: No module named 'ClassCosmicIntegrator'

## Create instance of ClassCosmicIntegrator

In [3]:
CI = ClassCosmicIntegrator.CosmicIntegrator(pathCOMPAS=pathData, fileName=fileName)

TypeError: __init__() got an unexpected keyword argument 'fileName'

## Set the COMPAS class in our cosmicIntegrator

In [4]:
# Set lower and upper mass limits used for Kroupa IMF when drawing ZAMS stars in the COMPAS run
CI.COMPAS.Mlower = 5.0
CI.COMPAS.Mupper = 150.0

CI.COMPAS.binaryFraction = 0.7

CI.COMPAS.setGridAndMassEvolved()
CI.COMPAS.setCOMPASDCOmask()  # By default, this masks for BBHs formed under the pessimistic CE assumption
CI.COMPAS.setCOMPASData()  # This carries out the mask

NameError: name 'CI' is not defined

## Specify the MSSFR model

In [ ]:
CI.MSSFR.SFRprescription = "Neijssel et al. (2019)"
CI.MSSFR.Zprescription = "logNormal"
CI.MSSFR.logNormalPrescription = "Neijssel Phenomenological"

# Set the metallicity grid from the COMPAS output
CI.MSSFR.metallicityGrid = CI.COMPAS.metallicityGrid
CI.MSSFR.calculateMetallicityBinEdges()
CI.calcBirthRedshift() 

## Perform the cosmic integration

In [ ]:
CI.cosmologicalIntegration()

## Create chirp mass distribution

In [ ]:
chirpMass = CI.COMPAS.mChirp
ratePerSys_observed = np.sum(CI.ratePerSys_observed, axis=0)
mask = chirpMass < 100 # Set maximum plotted chirp mass to be 100 Msun

# Set grid in chirp mass
binsMchirp = np.linspace(0, 40, 20)
dMchirp = np.diff(binsMchirp)
center = (binsMchirp[1:] + binsMchirp[:-1]) / 2.0

yvalues, _ = np.histogram(chirpMass[mask], bins=binsMchirp, weights=ratePerSys_observed[mask])
dydMchirp = np.divide(yvalues, dMchirp)

## Plot chirp mass distribution

In [ ]:
fig, axes = plt.subplots(1, 1)
axes.plot(center, dydMchirp)
plt.title("Chirp Mass Distribution of LIGO BBHs at O1 Sensitivity")
axes.set_xlabel("$\mathcal{M}_c}$ / $\mathrm{M}_\odot$")
axes.set_ylabel("$\\frac{d^2N}{dtd\mathcal{M}_c}$ / $\mathrm{yr}^{-1}\mathrm{M}_\odot^{-1}$")
plt.show()